In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [4]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [5]:
import pandas as pd
import pandas_profiling
data = pd.read_csv('~/2021_deep_learning_18/dataset/train_final_tokenized.csv')
pr=data.profile_report()
pr.to_file('pr_report.html')

Summarize dataset:   0%|          | 0/18 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
max(data['Token'].str.len())

304

In [21]:
data

,Id,Category,Sentence,Token,BERTToken
0,0,3,-LRB- The film -RRB- tackles the topic of rela...,"['film', 'tackles', 'topic', 'relationships', ...","[101, 1011, 1048, 15185, 1011, 1996, 2143, 101..."
1,1,2,"Lavishly , exhilaratingly tasteless .","['lavishly', 'exhilaratingly', 'tasteless']","[101, 22689, 2135, 1010, 4654, 26415, 15172, 2..."
2,2,4,It is also beautifully acted .,"['also', 'beautifully', 'acted']","[101, 2009, 2003, 2036, 17950, 6051, 1012, 102]"
3,3,1,"But , like Silence , it 's a movie that gets u...","['like', 'silence', ""'s"", 'movie', 'gets', 'sk...","[101, 2021, 1010, 2066, 4223, 1010, 2009, 1005..."
4,4,2,It 's been made with an innocent yet fervid co...,"[""'s"", 'made', 'innocent', 'yet', 'fervid', 'c...","[101, 2009, 1005, 1055, 2042, 2081, 2007, 2019..."
...,...,...,...,...,...
11539,11539,3,"Although Frailty fits into a classic genre , i...","['although', 'frailty', 'fits', 'classic', 'ge...","[101, 2348, 25737, 3723, 16142, 2046, 1037, 44..."
11540,11540,1,Mediocre fable from Burkina Faso .,"['mediocre', 'fable', 'burkina', 'faso']","[101, 19960, 3695, 16748, 28458, 2013, 23089, ..."
11541,11541,4,Like all great films about a life you never kn...,"['like', 'great', 'films', 'life', 'never', 'k...","[101, 2066, 2035, 2307, 3152, 2055, 1037, 2166..."
11542,11542,4,Those who are n't put off by the film 's auste...,"[""n't"", 'put', 'film', ""'s"", 'austerity', 'fin...","[101, 2216, 2040, 2024, 1050, 1005, 1056, 2404..."


In [7]:
def create_model(is_predicting, input_dis, input_mask, segment_ids, labels, num_labels):
    
    bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
    bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segement_ids=segement_ids)
    bert_outputs = bert_module(inputs=bert_inputs, signature="tokens", as_dict = True)
    
    output_layer = bert_outputs["poole_outputs"]
    hidden_size = output_layer.shape[-1].value
    
    output_weights = tf.get_variable("output_weights", [num_labes, hidden_size],
                                    initializer =tf.truncated_normal_initializer(stddev=0.02))
    
    output_bias = tf.get_variable("output_bias", [num_labels],
                                 initializer=tf.zeros_initializer())
    
    with tf.variable_scope("loss"):
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
        
        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=1)
        
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=float32)
        
        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        
        if is_predicting:
            return (predicted_labels, log_probs)
        
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)

In [8]:
def model_fn_builder (num_labels, learning_rate, num_train_steps, num_warmup_steps):
    def model_fn(features, labels, mode, params):
        
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segement_ids = features["segement_ids"]
        label_ids = features["label_ids"]
        
        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
        
        if not is_predicting:
            (loss, predicted_labels, log_probs) = create_model(
            s_predicting, input_dis, input_mask, segment_ids, labels, num_labels)
            
            train_op = bert.optimization.create_optimizer(
            loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu = False)   
            
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics,f1_score(label_ids, predicted_labels)
                return {"accuracy": accuracy, "f1_score": f1_score}
            
            eval_metrics = metric_fn(label_ids, predicted_labels)
            
            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_meric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(
            is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
            
            predictions = {
                'probabilities': log_probs,
                'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)
        
    return model_fn           

In [9]:
from sklearn.metrics import f1_score, accuracy_score

class Base:
    def __init__(self) -> None:
        pass
    
    def read_data(self, fname: str, lower_case: bool=False, 
                  colnames=['Category', 'Sentence', 'Token', 'BERTToken']) -> pd.DataFrame:
        
        df = pd.read_csv(fname, sep='\t', header = None, names=colnames)
        return df
    
    def accuracy(self, df: pd.DataFrame) -> None:
        acc = accuracy_score(df['Category'], df['pred'])*100
        f1 = f1_score(df['Category'], df['pred'], average='macro')
        print("Accuracy: {}\nMacro F1-score: {}".format(acc, f1))

In [10]:
class ExampleSentiment(Base):
    def __init__(self, model_file: str=None) -> None:
        super().__init__()
        
    def score(self, text: str) -> int:
        #sentiment scoring technique
        return 1
        
    def predict(self, train_file: None, test_file: str, lower_case: bool) -> pd.DataFrame:
        #DataFrame that applies the sentiment scoring method on each row of the test set
        df = self.read_data(test_file, lower_case)
        df['pred'] = df['text'].apply(self.score)

In [11]:
# BERT-Small
BATCH_SIZE = 8
LEARNING_RATE = 3e-4
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [12]:
num_train_steps = int(len(data) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [13]:
OUTPUT_DIR = '/output'
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [14]:
label_list = [1,2,3,4,5]
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1080361c18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:

from tensorflow import keras
import os
import re

# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df

In [16]:

train, test = download_and_load_datasets()

In [17]:
train = train.sample(5000)
test = test.sample(5000)

In [18]:
train.columns

Index(['sentence', 'sentiment', 'polarity'], dtype='object')

In [19]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

In [20]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [21]:
train_InputExamples

1375     <bert.run_classifier.InputExample object at 0x...
13246    <bert.run_classifier.InputExample object at 0x...
12326    <bert.run_classifier.InputExample object at 0x...
18341    <bert.run_classifier.InputExample object at 0x...
20838    <bert.run_classifier.InputExample object at 0x...
                               ...                        
20327    <bert.run_classifier.InputExample object at 0x...
24276    <bert.run_classifier.InputExample object at 0x...
15499    <bert.run_classifier.InputExample object at 0x...
5699     <bert.run_classifier.InputExample object at 0x...
12088    <bert.run_classifier.InputExample object at 0x...
Length: 5000, dtype: object

In [22]:
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [23]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [24]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 5000


INFO:tensorflow:Writing example 0 of 5000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)
